In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
import cv2
import skimage.exposure
from numpy.random import default_rng
import os
import random

In [3]:
import matplotlib.pyplot as plt

# Set default color map and interpolation method for images.
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'

%matplotlib inline

In [7]:
# Load and resize images
base_directory = "initial"
image_extensions = {".jpg", ".jpeg", ".png"}  # Supported extensions

# Collect all image file paths
all_images = []
for root, _, files in os.walk(base_directory):
    for file in files:
        if os.path.splitext(file)[1].lower() in image_extensions:
            all_images.append(os.path.join(root, file))

random.shuffle(all_images)

#Load the selected images using OpenCV
loaded_images = []
resized_images = [] 

for image_path in all_images:
    img = cv2.imread(image_path)  # Load the image
    if np.all(img[:, :, 0] == img[:, :, 1]) and np.all(img[:, :, 1] == img[:, :, 2]):
        pass #filter out grayscale images
    else:
        # Resize the image to 512x512 using cubic interpolation
        resized_img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)
        resized_images.append(resized_img)
        loaded_images.append(img)


In [8]:
#write the resize images into a new folder "paintings, with the names 0.jpg to max_number.jpg
for i in range(len(resized_images)):
    cv2.imwrite("paintings/"+str(i)+".jpg",resized_images[i])

In [9]:
#helper function
def display(figsize, images):
    """
    Display one or more images in a single figure.

    Parameters:
        - figsize: Size of the figure with (width, height).
        - images: List of tuples with (image, title, position).
    """
    plt.figure(figsize=figsize)

    for image, title, position in images:
        plt.subplot(*position if isinstance(position, tuple) else [position])
        plt.title(title)
        plt.axis('off')
        plt.imshow(image)

    plt.show()

In [10]:
#helper function for crack generation
def add_curvature(p1, p2, n_points=20, curve_intensity=10):
    """
    Adds curvature to a straight line between two points.
    - p1, p2: Endpoints of the line.
    - n_points: Number of intermediate points along the line.
    - curve_intensity: Amplitude of the random curvature.
    """
    # Generate a straight line
    x = np.linspace(p1[0], p2[0], n_points)
    y = np.linspace(p1[1], p2[1], n_points)
    
    # Add curvature using a sinusoidal or random perturbation
    mid_idx = n_points // 2
    for i in range(n_points):
        factor = np.sin((i - mid_idx) / n_points * np.pi)  # Sinusoidal factor for smooth curves
        x[i] += np.random.uniform(-curve_intensity, curve_intensity) * factor
        y[i] += np.random.uniform(-curve_intensity, curve_intensity) * factor

    return np.array([x, y]).T

In [11]:
#Creats a crack-like pattern using voronoi algorithm and adds curvature using the helper function
def generate_voronoi_cracks(width, height, num_points, line_thickness, curve_intensity, output_path):
    """
    Generates a crack-like pattern using Voronoi diagrams with curved edges and saves it as an image.
    - width, height: Dimensions of the canvas in pixels.
    - num_points: Number of random seed points.
    - line_thickness: Thickness of crack lines.
    - curve_intensity: Degree of curvature added to lines.
    - output_path: Path where the image will be saved.
    """
    # Initialize blank canvas
    canvas = np.ones((height, width, 4), dtype=np.uint8) * 255  # (height, width, 4 channels, white background)
    canvas[:, :, 3] = 0  # Set the alpha channel to 0 (transparent)
    
    # Generate random points
    points = np.random.rand(num_points, 2) * [width, height]
    
    # Create the Voronoi diagram
    vor = Voronoi(points)
    
    for ridge in vor.ridge_vertices:
        if -1 not in ridge:  # Ignore infinite ridges
            # Original line endpoints
            p1, p2 = vor.vertices[ridge]
            
            # Add curvature to the line
            curved_line = add_curvature(p1, p2, n_points=10, curve_intensity=curve_intensity)
            
            # Draw the curved crack on the canvas
            for i in range(len(curved_line) - 1):
                start = tuple(curved_line[i].astype(int))
                end = tuple(curved_line[i + 1].astype(int))
                cv2.line(canvas, start, end, color=(0, 0, 0,255), thickness=line_thickness)
    
    # Save the result
    cv2.imwrite(output_path, canvas)
    print(f"Crack pattern saved to {output_path}")
    return canvas

In [12]:
#Creats simple scratches
def generate_scratches(mask, overlay):
    height, width = mask.shape[:2]
    num_scratches= np.random.randint(1,6)
    
    colors = [(33, 67, 101),(0,0,0),(255,255,255)]
    
    for _ in range(num_scratches):
        base_color = colors[np.random.randint(len(colors))]
        alpha = np.random.randint(20,100)
        color = (*base_color, alpha)
        # Start at a random position
        x, y = np.random.randint(0, width), np.random.randint(0, height)
        num_segments = np.random.randint(1, 4)  # Number of segments per scratch
        thickness = np.random.randint(1, 6)  # Random line thickness
        
        # Generate a wavy path for the scratch
        for _ in range(num_segments):
            # Slightly vary the direction of the line
            x_offset = np.random.randint(-50, 50)
            y_offset = np.random.randint(-50, 50)
            x_new = np.clip(x + x_offset, 0, width - 1)
            y_new = np.clip(y + y_offset, 0, height - 1)
            
            # Draw on the mask (grayscale for damage detection)
            cv2.line(mask, (x, y), (x_new, y_new), 255, thickness)  # White line for the mask
            
            # Draw on the transparent overlay (colored scratch)
            cv2.line(overlay, (x, y), (x_new, y_new), color, thickness)  # Colored line for visual
        
            # Update the current position
            x, y = x_new, y_new
    
    return mask, overlay


In [13]:
#Creates rough vertical scratches/gekritzel
def generate_rough_scratches(mask,overlay):
    height, width = mask.shape[:2]
    colors = [(0, 0, 0), (255,255,255)]
    num_scratches = np.random.randint(1,2)
    for _ in range(num_scratches):
        
        base_color = colors[np.random.randint(len(colors))]
        alpha = np.random.randint(20,100)
        color = (*base_color, alpha)
        x, y = np.random.randint(0, width), np.random.randint(0, height)
        num_segments = np.random.randint(1, 3)  # Number of segments per scratch
        thickness = np.random.randint(10, 15)  # Approximate thickness
        
        for _ in range(num_segments):
            
            length = np.random.randint(30, 100)
            points = []
            for i in range(length):
                # Create points along the length of the scratch with random offsets
                x_offset = np.random.randint(-thickness // 2, thickness // 2)
                y_offset = i - length // 2
                points.append((x + x_offset, y + y_offset))
            
            # Convert points to a contour (polygon)
            points = np.array(points, dtype=np.int32)
            points = points.reshape((-1, 1, 2))  # Required format for cv2.drawContours
            
            # Draw the rough scratch as a filled polygon
            cv2.drawContours(mask, [points], -1, 255, -1)
            cv2.drawContours(overlay, [points], -1, color, -1)
            # Slightly move the starting point for the next segment
            x += np.random.randint(-50, 50)
            y += np.random.randint(-50, 50)
            
            # Clip to canvas boundaries
            x = np.clip(x, 0, width - 1)
            y = np.clip(y, 0, height - 1)
            
    return mask, overlay


In [14]:
#creats rough horizontal scratches/gekritzel
def generate_horizontal_damage(mask,overlay):
    height, width = mask.shape[:2]
    colors = [(0, 0, 0), (255,255,255)]
    num_damages = np.random.randint(1,3)
    for _ in range(num_damages):
        # Choose a random color
        base_color = colors[np.random.randint(len(colors))]
        alpha = np.random.randint(20,100)
        color = (*base_color, alpha)
        
        # Start at a random position
        x, y = np.random.randint(0, width), np.random.randint(0, height)
        num_segments = np.random.randint(1, 3)  # Number of segments per damage
        thickness = np.random.randint(10, 15)  # Approximate thickness
        
        for _ in range(num_segments):
            # Generate a rough contour for the horizontal damage
            length = np.random.randint(50, 150)
            points = []
            for i in range(length):
                # Create points along the width with random vertical offsets
                x_offset = i - length // 2
                y_offset = np.random.randint(-thickness // 2, thickness // 2)
                points.append((x + x_offset, y + y_offset))
            
            points = np.array(points, dtype=np.int32)
            points = points.reshape((-1, 1, 2))  # Required format for cv2.drawContours
            
            # Draw the rough damage as a filled polygon
            cv2.drawContours(mask, [points], -1, 255, -1)
            cv2.drawContours(overlay, [points], -1, color, -1)
            
            # Slightly move the starting point for the next segment
            x += np.random.randint(-50, 50)
            y += np.random.randint(-30, 30)
            
            # Clip to canvas boundaries
            x = np.clip(x, 0, width - 1)
            y = np.clip(y, 0, height - 1)
            
    return mask, overlay


In [15]:
#creats abrasions.
def generate_abrasions(mask, overlay):
    seedval = np.random.randint(1,100)
    rng = default_rng(seed=seedval)
    
    # define image size
    height, width = mask.shape[:2]
    
    # create random noise image
    noise = rng.integers(0, 255, (height,width), np.uint8, True)
    
    # blur the noise image to control the size
    blur = cv2.GaussianBlur(noise, (0,0), sigmaX=10, sigmaY=10, borderType = cv2.BORDER_DEFAULT)
    
    # stretch the blurred image to full dynamic range
    stretch = skimage.exposure.rescale_intensity(blur, in_range='image', out_range=(0,255)).astype(np.uint8)
    
    # threshold stretched image to control the size
    thresh = cv2.threshold(stretch, 200, 255, cv2.THRESH_BINARY)[1]
    
    # apply morphology open and close to smooth out shapes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    result = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel)
    mask[result ==255] = 255
    colors = [(0,0,0,255),(255,255,255,255)]
    color = colors[np.random.randint(len(colors))]
    overlay[result == 255] = color
    return mask, overlay

In [16]:
#function to load images from folder
def load_images(folder_path,grayscale=False):
    """
    Load all images from a flat folder without resizing.
    
    Args:
        folder_path (str): Path to the dataset folder containing .jpg images.
    
    Returns:
        list: List of images as NumPy arrays.
    """
    images = []
    image_names = []  # Optional: Store file paths for reference

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Check for valid image extensions
        # Load the image
        if grayscale:
            image = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
        else:
            image = cv2.imread(file_path)
        if image is not None:
            images.append(image)
            image_names.append(file_name)  # Optional: Track file paths
        else:
            print(f"Failed to load image: {file_path}")
            os.remove(file_path)

    print(f"Loaded {len(images)} images from {folder_path}")
    return images, image_names
folder_path = "paintings/"
images, file_names = load_images(folder_path)


Loaded 2741 images from paintings/


In [19]:
#generate cracks with the voronoi function (takes a bit)
cracks = []
for i in range(len(images)):
    h,w = images[i].shape[:2]
    num_points = 300
    line_thickness = 1    
    curve_intensity = 1
    output_path = 'inverted_crack_mask/' + file_names[i]
    crack = generate_voronoi_cracks(w, h, num_points, line_thickness, curve_intensity, output_path)
    cracks.append(crack)
    print(i,w,h)

Crack pattern saved to inverted_crack_mask/427.jpg
0 512 512
Crack pattern saved to inverted_crack_mask/1702.jpg
1 512 512
Crack pattern saved to inverted_crack_mask/1906.jpg
2 512 512
Crack pattern saved to inverted_crack_mask/2237.jpg
3 512 512
Crack pattern saved to inverted_crack_mask/2518.jpg
4 512 512
Crack pattern saved to inverted_crack_mask/2394.jpg
5 512 512
Crack pattern saved to inverted_crack_mask/114.jpg
6 512 512
Crack pattern saved to inverted_crack_mask/1421.jpg
7 512 512
Crack pattern saved to inverted_crack_mask/965.jpg
8 512 512
Crack pattern saved to inverted_crack_mask/13.jpg
9 512 512
Crack pattern saved to inverted_crack_mask/371.jpg
10 512 512
Crack pattern saved to inverted_crack_mask/1543.jpg
11 512 512
Crack pattern saved to inverted_crack_mask/912.jpg
12 512 512
Crack pattern saved to inverted_crack_mask/611.jpg
13 512 512
Crack pattern saved to inverted_crack_mask/808.jpg
14 512 512
Crack pattern saved to inverted_crack_mask/20.jpg
15 512 512
Crack pattern

In [23]:
#apply the crack mask seperatly, since it needs different alpha values.
damaged_images = []
for i in range(len(images)):
    image_rgb = images[i]  # Shape (height, width, 3)
    crack_rgb = cracks[i][:, :, :3]  # Extract RGB part of cracks (Shape: height, width, 3)
    
    # Extract alpha channel from cracks (Shape: height, width, 1)
    crack_alpha = cracks[i][:, :, 3]  
    
    # Convert alpha to a range of 0 to 1 for blending (alpha/255)
    crack_alpha_normalized = crack_alpha.astype(np.float32) / 255.0
    alpha_value = np.random.randint(5,25)
    alpha_blend = alpha_value / 100.0
    
    # Perform the blending only where crack_alpha is non-zero
    blended_rgb = image_rgb * (1 - alpha_blend * crack_alpha_normalized[..., None]) + crack_rgb * alpha_blend * crack_alpha_normalized[..., None]
    
    # Clip values to stay within [0, 255] and convert back to uint8
    blended_rgb = np.clip(blended_rgb, 0, 255).astype(np.uint8)

    # Stack the blended RGB with the original alpha channel to create the final RGBA image
    blended = np.dstack((blended_rgb, crack_alpha))  # Add alpha channel back
    
    # Append to the list of damaged images
    damaged_images.append(blended)
    
    # Save the blended image (RGBA)
    cv2.imwrite(f"damaged_paintings/{file_names[i]}", blended)

    
    #invert the crack_map for later
    cv2.imwrite("crack_mask/"+file_names[i], 255-cracks[i])
    
print("done")
    

done


In [22]:
#next we wanna add some other damages. we just add them to the image and the mask.
folder_path = "crack_mask/"
masks, file_names = load_images(folder_path,grayscale=True)
damage_functions = [generate_scratches,generate_rough_scratches,generate_horizontal_damage,generate_abrasions]
for i in range(len(masks)):
    mask = masks[i]
    height,width = mask.shape[:2]
    overlay = np.zeros((height, width, 4), dtype=np.uint8)
    num_functions = np.random.randint(1,4)
    selected_functions = np.random.choice(damage_functions, num_functions, replace=False)
    for func in selected_functions:
        mask,overlay = func(mask,overlay)
    cv2.imwrite("final_mask/"+file_names[i],mask)
    
    image = damaged_images[i]
    image = image[:,:,:3]
    overlay_rgb = overlay[:,:,:3]
    overlay_alpha = overlay[:,:,3]
    alpha_normalized = overlay_alpha.astype(float) / 255.0
    blended_image = image.astype(float) * (1 - alpha_normalized[:, :, None]) + overlay_rgb.astype(float) * alpha_normalized[:, :, None]
    cv2.imwrite("final_paintings/"+file_names[i],blended_image)
plt.imshow(overlay)
plt.show()


Loaded 2759 images from crack_mask/


IndexError: list index out of range